In [3]:
import os

In [4]:
!tree ../dataset

../dataset
├── en_espanol
│   ├── docx2txt.py
│   ├── Второй_жанр_исходная.txt
│   └── Первый_жанр_исходная.txt
├── Второй_жанр_исходная.txt
├── Первый_жанр_исходная.txt
├── Сокращение по частям речи
│   ├── 1.Первый жанр исходная выборка.txt
│   ├── 2.Первый жанр без клауз, включающих наречия.txt
│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
│   ├── 4. Первый жанр без клауз, включающих глаголы и наречия.txt
│   ├── Без прилагательных второй жанр.txt
│   ├── Без прилагательных первый жанр.txt
│   └── Случайные выборки.txt
└── сокращение по частотности
    ├── 1а_ без сокращений.txt
    ├── 1б_Изъяты лексемы с частотой выше 100.txt
    ├── 1в_Изъяты лексемы с частотой выше 49.txt
    ├── 1г_Изъяты лексемы с частотой выше 29.txt
    ├── 1д_Изъяты лексемы с частотой выше 9.txt
    ├── 2а_ без сокращений.txt
    ├── 2б _ Изъяты лексемы с частотой выше 100..txt
    ├── 2в_ Изъяты лексемы с частотой выше 49..txt
    ├── 2г_  Изъяты лексемы с частотой выше 29.txt
    └── 2д_ Изъяты 

In [5]:
import re
from nltk.util import ngrams
from nltk.tokenize import word_tokenize, sent_tokenize

def segment_text_with_ngrams(text, n=3):  
    """Segmenta el texto dividiéndolo en frases, evitando dividir dentro de paréntesis."""
    
    def replace_inside_parentheses(match):
        """Reemplaza los signos de puntuación dentro de los paréntesis para evitar segmentación incorrecta."""
        content = match.group(0)
        content = content.replace(".", "§").replace("!", "§").replace("?", "§")
        return content  # Devuelve el contenido modificado
    
    text = re.sub(r'\([^()]*\)', replace_inside_parentheses, text)  # Evita problemas con paréntesis anidados

    sentences = sent_tokenize(text, language="russian")  # Segmentar en frases correctamente

    segmented_sentences = []
    for sentence in sentences:
        sentence = sentence.replace("§", ".")  # Restaurar puntuación dentro de paréntesis
        words = word_tokenize(sentence, language="russian")  # Tokenización de palabras
        if words:
            segmented_sentences.append(" ".join(words))

    return [s.strip() for s in segmented_sentences if s.strip()]

def clean_text(text):
    """Elimina texto entre corchetes y caracteres innecesarios."""
    cleaned_text = re.sub(r'\[.*?\]', '', text)  # Eliminar contenido entre corchetes []
    return cleaned_text.strip()

def load_texts(filename, label):
    with open(filename, "r", encoding="utf-8") as f:
        text = f.read()
    
    sentences = segment_text_with_ngrams(text)  
    cleaned_sentences = [clean_text(sentence) for sentence in sentences]
    
    return [{"text": sentence, "label": label} for sentence in cleaned_sentences if sentence]


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar ambos géneros
genre1 = load_texts("../dataset/Первый_жанр_исходная.txt", label="0")
genre2 = load_texts("../dataset/Второй_жанр_исходная.txt", label="1")
# Combinar ambos géneros en un DataFrame
all_texts_df = pd.DataFrame(genre1 + genre2)

# Eliminar filas con valores NaN o vacíos en la columna 'text'
all_texts_df = all_texts_df[all_texts_df['text'].notna()]
all_texts_df = all_texts_df[all_texts_df['text'] != '']

# Dividir nuevamente en train, validation y test
train_df, test_df = train_test_split(all_texts_df, test_size=0.2, random_state=42, stratify=all_texts_df['label'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])

# Guardar los conjuntos limpios
all_texts_df.to_csv("data.csv",index=False)
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)

print(f"Tamaño del conjunto completo: {len(all_texts_df)}")
print(f"Tamaño del conjunto de entrenamiento: {len(train_df)}")
print(f"Tamaño del conjunto de validación: {len(val_df)}")
print(f"Tamaño del conjunto de prueba: {len(test_df)}")

Tamaño del conjunto completo: 908
Tamaño del conjunto de entrenamiento: 580
Tamaño del conjunto de validación: 146
Tamaño del conjunto de prueba: 182


In [7]:
import pandas as pd

# Cargar los datos desde los archivos CSV
all_texts_df = pd.read_csv("data.csv")
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")
test_df = pd.read_csv("test.csv")

# Mostrar las primeras filas de cada DataFrame
print("Datos completos:")
print(all_texts_df.head())

print("\nConjunto de entrenamiento:")
print(train_df.head())

print("\nConjunto de validación:")
print(val_df.head())

print("\nConjunto de prueba:")
print(test_df.head())

# Mostrar información general sobre los DataFrames
print("\nInformación de los DataFrames:")
print("\nDatos completos:")
print(all_texts_df.info())

print("\nConjunto de entrenamiento:")
print(train_df.info())

print("\nConjunto de validación:")
print(val_df.info())

print("\nConjunto de prueba:")
print(test_df.info())

# Mostrar estadísticas descriptivas de los datos numéricos (si los hay)
print("\nEstadísticas descriptivas:")
print("\nDatos completos:")
print(all_texts_df.describe())

print("\nConjunto de entrenamiento:")
print(train_df.describe())

print("\nConjunto de validación:")
print(val_df.describe())

print("\nConjunto de prueba:")
print(test_df.describe())

Datos completos:
                                                text  label
0                  Кустарник до 1.5 метров высотой .      0
1                           Ветви темно-коричневые .      0
2  Годовалые побеги утолщенные , короткие , темно...      0
3  Почки мономорфные , 4.5-9 мм длиной , яйцевидн...      0
4  Прилистники 1.5-12 мм длиной , полусердцевидны...      0

Conjunto de entrenamiento:
                                                text  label
0  Ведь психика – это не просто система , а супер...      1
1                          Темя с поперечным килем .      0
2  Цвет в образцах и зернах под бинокуляром ярко-...      0
3  На втором членике лапок 15-24 волоска разной д...      0
4                                            Самец .      0

Conjunto de validación:
                                                text  label
0  Строение тела со следующими особенностями : на...      0
1  Взаимодействие между популяциями есть прежде в...      1
2            Спайность или отд